# 6주차 크롤링 과제

## 과제 1
- ### TO - DO : Selenium을 이용한 인스타 크롤링 수도코드 작성
 - 상세 : 5,000 개의 게시글 정보 : 내용,날짜, 댓글내용, 댓글계정

In [589]:
import selenium 
from selenium import webdriver
import random
from time import sleep
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [3]:
post_contents=[]
post_date=[]
comment_contents=[]
comment_writer=[]

insta_url = "https://www.instagram.com" # 인스타 url 
browser = webdriver.Chrome() #브라우저 on
browser.get(insta_url) # 접속

In [ ]:
for scroll in range(1,100) : # 스크롤 한번 할때마다 50개씩 늘어난다고 가정
    # 게시물 태그 추출
    post_tags = browser.find_elements_by_class_name('posts_tag')
    # 게시물 정보 추출
    for post in post_tags :
        post_contents.append( (post.find_element_by_tag_name('contents')).text )
        post_date.append( (post.find_element_by_tag_name('d')).text )
        # 게시물 클릭
        post_button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div[4]/div[2]')
        post_button.click()
        # 댓글 태그 추출
        comment_tags= browser.find_elements_by_class_name('comments_tag')
        # 댓글 정보 추출
        for comment in comment_tags:
            comment_contents.append( (comment.find_element_by_tag_name('comment_contents')).text )
            comment_writer.append( (comment.find_element_by_tag_name('wrt')).text)
    
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤 끝까지
    sleep(random.uniform(0,1)) # 휴식

## 과제 2
- ### TO - DO : 다음 영화 사이트 2년치 데이터 크롤링
 - 상세 : 제목, 네티즌 평점, 평론가 평점, 누적 관객수, 누락데이터 적절한 처리, 
 - 채점기준 : 모듈화, End-to-End

### 1. 동적 크롤링 (Selenium)

In [681]:
def daum_movie_crawler_selenium():
    
    #변수입력받기(다양한 방식 시도)
    this_year=int(input('몇 년도부터 수집할까요?'))
    for_year=int(input('지금부터 과거 몇 년치 데이터를 수집해야하나요?'))
    
    #리스트 생성
    titles=[]
    net_grade=[]
    cri_grade=[]
    aud_list=[]
    
    #브라우저 실행
    main_url = 'https://movie.daum.net/boxoffice/yearly?year='+str(this_year)
    browser = webdriver.Chrome()
    browser.get(main_url)
    reg=re.compile('\d') #  수 추출을 위한 정규표현식    
    
    for times in range(for_year):
        links=[]
        poster_table=browser.find_elements_by_class_name('wrap_movie') # 포스터 테이블
        exist_verify_table=browser.find_elements_by_class_name('img_g') # 정보 존재 검정용 테이블
        
        for poster,exist_verify in zip(poster_table,exist_verify_table):
            
            # 정보 담고있는 테이블
            info_table = poster.find_element_by_tag_name('a')
            
            # 존재 검정용 변수
            ex_num=exist_verify.get_attribute('src')
            
            # 제목 추출 
            title = info_table.text
            titles.append(title)
            
            # 검정
            if len((str(ex_num))) <= 15: 
                net_grade.append(None)
                cri_grade.append(None)
                links.append(None)
                continue
            
            # 평점 테이블
            score_table= reg.findall(poster.find_element_by_class_name('info_grade').text)
            
            # 네티즌 평점
            net_score = float(score_table[1])+float(score_table[3])/10
            net_grade.append(net_score)
            
            # 평론가 평점
            try :
                cri_score = float(score_table[5])+float(score_table[7])/10
                cri_grade.append(cri_score)
            except Exception as e :
                cri_grade.append(None)
                print(e,'Error at'+str(title))
                
             # 링크 추출
            link = info_table.get_attribute('href')
            links.append(link)
        
        for link in links:
            try :
                browser.get(link) # 이동
                aud_list.append(int(''.join(reg.findall(browser.find_element_by_id('totalAudience').text))))
            except Exception as e :
                aud_list.append(None)
            
        browser.get(main_url)
        button=browser.find_element_by_xpath('//*[@id="divCalendarBox"]/a')
        button.click()
            
    data = pd.concat([pd.Series(titles),pd.Series(net_grade),pd.Series(cri_grade),pd.Series(aud_list)],axis=1)            
    data.columns=['title','netizen_score','critic_grade','totalAudience']
    return data
        
        

In [682]:
df_s=daum_movie_crawler_selenium()

지금은 몇 년도 인가요?2019
지금부터 과거 몇 년치 데이터를 수집해야하나요?2
list index out of range Error at극한직업
list index out of range Error at어벤져스: 엔드게임
list index out of range Error at알라딘
list index out of range Error at기생충
list index out of range Error at스파이더맨: 파 프롬 홈
list index out of range Error at엑시트
list index out of range Error at캡틴 마블
list index out of range Error at라이온 킹
list index out of range Error at봉오동 전투
list index out of range Error at토이 스토리 4
list index out of range Error at돈
list index out of range Error at악인전
list index out of range Error at분노의 질주: 홉스&쇼
list index out of range Error at증인
list index out of range Error at사바하
list index out of range Error at알리타: 배틀 엔젤
list index out of range Error at뺑반
list index out of range Error at주먹왕 랄프 2: 인터넷 속으로
list index out of range Error at걸캅스
list index out of range Error at사자
list index out of range Error at드래곤 길들이기 3
list index out of range Error at나의 특별한 형제
list index out of range Error at어스
list index out of range Error at생일
list index out of range 

## 2. 정적 크롤링(Request)

In [3]:
from time import sleep
import re
from tqdm import tqdm
from random import randint
from bs4 import BeautifulSoup as bs
import requests
import lxml.html

In [2]:
def compute_score(table):
    score = table.findAll('span',{"class" : 'num_grade'})
    first=float(score[0].text)
    second=float(score[2].text)/10
    return (first+second)

In [687]:
def daum_movie_crawler(start_year,end_year):
    #추출할 데이터 리스트생성
    titles=[]
    net_grade=[]
    cri_grade=[]
    aud_list=[]
    
    for year in range(int(start_year),int(end_year+1)):
        #원활한 크롤링을 위한 헤더
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        #url 입력
        main_url = 'https://movie.daum.net/boxoffice/yearly?year='+str(year)
        response = requests.get(main_url,headers=headers) #접속
    
        #테이블 분리
        body_table = bs(response.text,'lxml')
        poster_table = body_table.find_all('div',class_='wrap_movie') # 포스터 테이블  
        exist_verify_table=body_table.find_all(class_='img_g') # 정보가 존재하는지 검정용 테이블

        reg=re.compile('\d') #  관객수 추출을 위한 정규표현식
        reg2=re.compile('src.*/') # 이미지 링크가 없는경우 다른 정보도 없는 것 발견, 따라서 링크 부분의 길이가 짧으면 
                                  # 변수를 NA값으로 채워서 용이한 df생성 의도
    
    
        # 제목 , 네티즌 평점, 평론가 평점 추출
        for poster,exist_verify in zip(poster_table,exist_verify_table):
            # 제목 추출 
            title = poster.find('a', class_='name_movie #title').text
            titles.append(title)
            # 검정
            if len(str(reg2.findall(str(exist_verify)))) <= 15: 
                net_grade.append(None)
                cri_grade.append(None)
                aud_list.append(None)
                continue
            
            # 네티즌 평점
            net_grade_table = poster.find("span",{"class":"wrap_grade grade_netizen"})
            net_grade.append(compute_score(net_grade_table))
            
            # 평론가 평점
            try : 
                critic_grade_table= poster.find("span",{"class":"wrap_grade grade_critic"})
                cri_grade.append(compute_score(critic_grade_table))
            except Exception as e :
                cri_grade.append(None)
                print(e,'Error at {} for {}'.format(year,title))
                
            # 누적 관객수
            
            movie_id=(''.join(reg.findall(poster.find('a')['href'])))        
            aud_url='https://movie.daum.net/moviedb/main/totalAudience.json?movieId='+movie_id # 링크 확보
            response = requests.get(aud_url,headers=headers)
            try : 
                aud_list.append(int(''.join(reg.findall(response.text))))
            except Exception as e :
                aud_list.append(None)
                print(e,str(movie_id)+'에서 누적관객추출 실패')
                
            
    data = pd.concat([pd.Series(titles),pd.Series(net_grade),pd.Series(cri_grade),pd.Series(aud_list)],axis=1)            
    data.columns=['title','netizen_score','critic_grade','totalAudience']
    return data

In [688]:
df=daum_movie_crawler(2018,2019)

'NoneType' object has no attribute 'findAll' Error at 2019 for 극한직업
'NoneType' object has no attribute 'findAll' Error at 2019 for 어벤져스: 엔드게임
'NoneType' object has no attribute 'findAll' Error at 2019 for 알라딘
'NoneType' object has no attribute 'findAll' Error at 2019 for 기생충
'NoneType' object has no attribute 'findAll' Error at 2019 for 스파이더맨: 파 프롬 홈
'NoneType' object has no attribute 'findAll' Error at 2019 for 엑시트
'NoneType' object has no attribute 'findAll' Error at 2019 for 캡틴 마블
'NoneType' object has no attribute 'findAll' Error at 2019 for 라이온 킹
'NoneType' object has no attribute 'findAll' Error at 2019 for 봉오동 전투
'NoneType' object has no attribute 'findAll' Error at 2019 for 토이 스토리 4
'NoneType' object has no attribute 'findAll' Error at 2019 for 돈
'NoneType' object has no attribute 'findAll' Error at 2019 for 악인전
'NoneType' object has no attribute 'findAll' Error at 2019 for 분노의 질주: 홉스&쇼
'NoneType' object has no attribute 'findAll' Error at 2019 for 증인
'NoneType' object has no a

In [697]:
df.sort_values(by='totalAudience',inplace=True,ascending=False)
df_s.sort_values(by='totalAudience',inplace=True,ascending=False)

In [706]:
display(df.info())
display(df_s.info()) # 구조, 결측치 동일 
# 결측치는 데이터타입을 float이나 int로 만들기 위해 None을 넣어주었음

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 50 to 15
Data columns (total 4 columns):
title            100 non-null object
netizen_score    99 non-null float64
critic_grade     56 non-null float64
totalAudience    99 non-null float64
dtypes: float64(3), object(1)
memory usage: 3.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 65
Data columns (total 4 columns):
title            100 non-null object
netizen_score    99 non-null float64
critic_grade     56 non-null float64
totalAudience    99 non-null float64
dtypes: float64(3), object(1)
memory usage: 3.9+ KB


None

In [710]:
df.to_csv('과제.csv',index=False)